In [11]:
import pandas as pd
import numpy as np

import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from math import exp

from statsmodels.tsa.holtwinters import ExponentialSmoothing
import re

import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning
warnings.simplefilter("ignore", ConvergenceWarning)
warnings.filterwarnings("ignore", message="No frequency information was provided")

In [26]:
# imports including those initially done from R

panel = pd.read_csv("commodities_panel.csv", parse_dates=["date"])
scales = pd.read_csv("series_scales.csv")
id_map = pd.read_csv("series_id_map.csv")
ends_early = pd.read_csv("ends_early.csv", parse_dates=["date"])
starts_late= pd.read_csv("starts_late.csv", parse_dates=["date"])
gaps_comm  = pd.read_csv("gaps_comm.csv", parse_dates=["date"])
commodities_template = pd.read_csv("submission_template_commodities.csv")
clean_comm_strict = pd.read_csv("clean_comm_strict.csv")
submission = pd.read_csv("submission_template_commodities.csv")

# keep only needed cols in panel; ensure types
panel = panel[["Commodity","flow","date","value","y"]].copy()
panel["flow"] = panel["flow"].astype("category")

print("Rows:", len(panel), "| series:", panel[["Commodity","flow"]].drop_duplicates().shape[0])
print(panel.head(3))

# check every series spans 2015-2023 monthly
grp = panel.sort_values("date").groupby(["Commodity","flow"]).agg(start=("date","min"), end=("date","max"), n=("date","size")).reset_index()
print("Min start:", grp["start"].min(), "Max end:", grp["end"].max())


Rows: 145800 | series: 1350
                   Commodity    flow       date       value          y
0  111 Agricultural Products  export 2015-01-01  6621741375  22.613624
1  111 Agricultural Products  export 2015-02-01  5957020004  22.507836
2  111 Agricultural Products  export 2015-03-01  5782566601  22.478113
Min start: 2015-01-01 00:00:00 Max end: 2023-12-01 00:00:00


/var/folders/6z/9dzyg6s964d4mtqp3fy4sxtr0000gn/T/ipykernel_74944/1945750918.py:21: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grp = panel.sort_values("date").groupby(["Commodity","flow"]).agg(start=("date","min"), end=("date","max"), n=("date","size")).reset_index()


NOTE: Initial EDA which includes dividing the data into different types of data (clean commodities, commodities with gaps, etc) was done in R and all are imported here so the EDA is not included in this notebook but instead in the R file

## Modeling for commodities with no problems (clean_comm_strict)

#### First dataset modeling (clean_comm_strict)
First of all, we are going to compare ETS model vs CNN-LSTM model. To do this, we will be doing rolling folds
That means, we will train on 4 different folds and get the average weighted metrics with the later years holding more weight as it is close to 2024 at the end;
1. Train on 2015 - 2019, predict 2020 (weight = 0.1)
2. Train on 2015 - 2020, predict 2021 (weight = 0.2)
3. Train on 2015 - 2021, predict 2022 (weight = 0.3)
4. Train on 2015 - 2022, predict 2023 (weight = 0.4)

We report sMAPE (scale-free) and MASE (≤1 means beating seasonal-naive). Model selection uses recency-weighted fold scores as mentioned.

At the end, we will then compare, for each commodity, which model did better overall. For whichever model did better for that commodity, it will be used to than train on 2015-2023 and predict the 2024 which will be used for the final submission.

#### Why these two models (ETS vs CNN-LSTM)

1. ETS, Error Trend and Seasonality with additive seasonality + damped trend
Chosen because monthly trade data has strong 12-month seasonality and occasional level/trend shifts. ETS models these components explicitly, needs minimal tuning, is stable with ~108 points, and is interpretable—a reliable baseline that tells us when simple seasonal structure already explains the data.

2. CNN-LSTM (L=24 -> H=12)
Chosen to capture nonlinear seasonal motifs (via CNN) and medium-term dependencies (via LSTM) that ETS can miss. On clean, gap-free series, deep sequence models often show better out-of-sample accuracy, which we’ll verify with rolling folds. The design uses lags only (no leakage) and forecasts the full 12-month horizon directly.

Bottom line: ETS gives a strong, low-variance, interpretable benchmark; CNN-LSTM tests for additional predictive signal (nonlinear/long-memory effects). We keep both, compare them fairly with rolling year-ahead CV, and then use the per-country, per-series winner for the final 2024 forecast.

ETS Model

In [12]:
EXPORT_COL = "export_value"
IMPORT_COL = "import_value"
FOLDS = [2020, 2021, 2022, 2023]
M = 12  # monthly seasonality

def to_ts(df_slice, value_col):
    # use existing monthly date column; enforce month-start frequency
    s = (df_slice.sort_values(["Year","month_num"])
         .set_index(pd.to_datetime(df_slice["date"]))
         [value_col].astype(float))
    return s.asfreq("MS")

def smape(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    denom = np.abs(y_true) + np.abs(y_pred)
    denom = np.where(denom == 0, 1.0, denom)
    return 200.0 * np.mean(np.abs(y_pred - y_true) / denom)

def mase(y_true, y_pred, insample, m=M):
    insample = np.asarray(insample)
    if len(insample) <= m: return np.nan
    scale = np.mean(np.abs(insample[m:] - insample[:-m]))
    if scale == 0: return np.nan
    return np.mean(np.abs(np.asarray(y_true) - np.asarray(y_pred))) / scale

def ets_fold_metrics(df_slice, value_col):
    s = to_ts(df_slice, value_col)
    rows = []
    for year in FOLDS:
        train = s[s.index.year <= (year-1)]
        test  = s[s.index.year == year]
        if len(test) != 12 or len(train) < 2*M:
            rows.append(dict(Year=year, sMAPE=np.nan, MASE=np.nan, used="skip"))
            continue
        model = ExponentialSmoothing(
            train,
            trend="add", damped_trend=True,
            seasonal="add", seasonal_periods=M,
            initialization_method="estimated"
        ).fit(optimized=True)
        fc = model.forecast(12).values
        damped = bool(getattr(model.model, "damped", False))
        rows.append(dict(
            Year=year,
            sMAPE=smape(test.values, fc),
            MASE=mase(test.values, fc, insample=train.values, m=M),
            used=f"ETS(add,add,damped={damped})"
        ))
    out = pd.DataFrame(rows)
    out["Series"] = value_col
    return out

# Run CV over all Commodities 
ets_cv_results_comm = []
for commodity, g in clean_comm_strict.groupby("Commodity"):
    for col in (EXPORT_COL, IMPORT_COL):
        res = ets_fold_metrics(g, col)
        res["Commodity"] = commodity
        ets_cv_results_comm.append(res)

ets_cv_results_comm = pd.concat(ets_cv_results_comm, ignore_index=True)
print("ETS CV (clean_comm_strict by Commodity):", ets_cv_results_comm.shape)
display(ets_cv_results_comm.head())

ETS CV (clean_comm_strict by Commodity): (5400, 6)


,Year,sMAPE,MASE,used,Series,Commodity
0,2020,13.584545,1.244165,"ETS(add,add,damped=False)",export_value,111 Agricultural Products
1,2021,20.611011,1.745365,"ETS(add,add,damped=False)",export_value,111 Agricultural Products
2,2022,7.124138,0.568771,"ETS(add,add,damped=False)",export_value,111 Agricultural Products
3,2023,26.504600,1.884077,"ETS(add,add,damped=False)",export_value,111 Agricultural Products
4,2020,4.327628,0.985434,"ETS(add,add,damped=False)",import_value,111 Agricultural Products


CNN-LSTM Model

We set L=24 (2 seasonal cycles) and H=12 to match the task horizon. All scaling is fit on train only per fold and windows are chronological so there is no leakage

In [13]:
M = 12
L, H = 24, 12
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
EXPORT_COL = "export_value"
IMPORT_COL = "import_value"
FOLDS = [2020, 2021, 2022, 2023]


def to_ts(df_slice, value_col):
    s = (df_slice.sort_values(["Year","month_num"])
         .set_index(pd.to_datetime(df_slice["date"]))
         [value_col].astype(float)).asfreq("MS")
    return s

def smape(y_true, y_pred, eps=1e-9):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    denom = np.abs(y_true) + np.abs(y_pred) + eps
    return float(np.mean(2*np.abs(y_pred - y_true) / denom))

def mase(y_true, y_pred, insample, m=M):
    insample = np.asarray(insample)
    if len(insample) <= m: return np.nan
    scale = np.mean(np.abs(insample[m:] - insample[:-m]))
    if scale == 0: return np.nan
    return float(np.mean(np.abs(np.asarray(y_true) - np.asarray(y_pred))) / scale)

def make_supervised(arr, lookback=L, horizon=H):
    X, Y = [], []
    T = len(arr)
    last_start = T - lookback - horizon
    if last_start < 0:
        return np.empty((0, lookback, 1)), np.empty((0, horizon))
    for t in range(last_start + 1):
        X.append(arr[t:t+lookback])
        Y.append(arr[t+lookback:t+lookback+horizon])
    X = np.asarray(X)[..., None] # (N, L, 1)
    Y = np.asarray(Y) # (N, H)
    return X, Y

class WindowDataset(Dataset):
    def __init__(self, X, Y):
        self.X = torch.from_numpy(X).float()
        self.Y = torch.from_numpy(Y).float()
    def __len__(self): return self.X.shape[0]
    def __getitem__(self, i): return self.X[i], self.Y[i]

# CNN LSTM Model
class CNNLSTMForecaster(nn.Module):
    def __init__(self, L=L, H=H, in_feats=1, conv_filters=32, k=5, lstm_units=64, dropout=0.2):
        super().__init__()
        self.conv1 = nn.Conv1d(in_channels=in_feats, out_channels=conv_filters, kernel_size=k, padding="same")
        self.act1 = nn.ReLU()
        self.conv2 = nn.Conv1d(in_channels=conv_filters, out_channels=conv_filters, kernel_size=k, padding="same")
        self.act2 = nn.ReLU()
        self.lstm = nn.LSTM(input_size=conv_filters, hidden_size=lstm_units, batch_first=True)
        self.drop = nn.Dropout(dropout)
        self.fc = nn.Linear(lstm_units, H)
    def forward(self, x):  # x: [B, L, F]
        x = x.transpose(1,2) # [B, F, L] for Conv1d
        x = self.act1(self.conv1(x)) # [B, C, L]
        x = self.act2(self.conv2(x)) # [B, C, L]
        x = x.transpose(1,2) # [B, L, C]
        x, _ = self.lstm(x) # [B, L, U]
        x = x[:, -1, :] # [B, U]
        x = self.drop(x)
        return self.fc(x) # [B, H]

def train_earlystop(model, train_loader, val_loader, epochs=200, lr=1e-3, patience=20):
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = nn.L1Loss()  # MAE tends to align with sMAPE goals
    best = float("inf"); best_state=None; wait=0
    for ep in range(1, epochs+1):
        model.train(); tr_loss=0.0; ntr=0
        for xb, yb in train_loader:
            xb, yb = xb.to(DEVICE), yb.to(DEVICE)
            opt.zero_grad()
            loss = loss_fn(model(xb), yb)
            loss.backward(); opt.step()
            tr_loss += loss.item()*xb.size(0); ntr += xb.size(0)
        model.eval(); va_loss=0.0; nva=0
        with torch.no_grad():
            for xb, yb in val_loader:
                xb, yb = xb.to(DEVICE), yb.to(DEVICE)
                va_loss += loss_fn(model(xb), yb).item()*xb.size(0); nva += xb.size(0)
        va = va_loss / max(1, nva)
        if va < best - 1e-6:
            best = va; best_state = {k:v.detach().cpu().clone() for k,v in model.state_dict().items()}; wait=0
        else:
            wait += 1
            if wait >= patience: break
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

# Fold evaluator (per Country x Series)
def dl_fold_metrics_commodity_series(df_slice, value_col, lookback=L, horizon=H):
    s = to_ts(df_slice, value_col)
    y = s.values
    y_log = np.log1p(np.clip(y, a_min=0, a_max=None))
    rows = []
    for year in FOLDS:
        train_idx = s.index.year <= (year-1)
        test_idx = s.index.year == year
        y_tr_log = y_log[train_idx]
        y_te = y[test_idx]
        if (np.sum(test_idx) != horizon) or (len(y_tr_log) < lookback + horizon):
            rows.append(dict(Year=year, sMAPE=np.nan, MASE=np.nan, used="skip"))
            continue

        mu, sd = float(y_tr_log.mean()), float(y_tr_log.std(ddof=0))
        if sd == 0: sd = 1.0
        y_tr_std = (y_tr_log - mu) / sd

        X, Y = make_supervised(y_tr_std, lookback=lookback, horizon=horizon)
        if X.shape[0] < 8:
            rows.append(dict(Year=year, sMAPE=np.nan, MASE=np.nan, used="insufficient_windows"))
            continue

        n = X.shape[0]; split = max(1, int(np.floor(0.9*n)))
        X_tr, Y_tr = X[:split], Y[:split]
        X_va, Y_va = X[split:], Y[split:]

        train_loader = DataLoader(WindowDataset(X_tr, Y_tr), batch_size=64, shuffle=True)
        val_loader = DataLoader(WindowDataset(X_va, Y_va), batch_size=64, shuffle=False)

        torch.manual_seed(42)
        model = CNNLSTMForecaster(L=lookback, H=horizon, in_feats=1,
                                  conv_filters=32, k=5, lstm_units=64, dropout=0.2).to(DEVICE)
        model = train_earlystop(model, train_loader, val_loader, epochs=200, lr=1e-3, patience=20)

        last_win_std = ((y_tr_log[-lookback:] - mu) / sd).reshape(1, lookback, 1)
        with torch.no_grad():
            pred_std = model(torch.from_numpy(last_win_std).float().to(DEVICE)).cpu().numpy().ravel()

        pred_log = pred_std * sd + mu
        pred_usd = np.expm1(pred_log).clip(min=0)

        rows.append(dict(
            Year=year,
            sMAPE=smape(y_te, pred_usd),
            MASE=mase(y_te, pred_usd, insample=y[train_idx], m=M),
            used="CNN-LSTM"
        ))
    out = pd.DataFrame(rows)
    out["Series"] = value_col
    return out

# Run CV over all perfect countries commodities 
dl_cv_results_comm = []
for commodity, g in clean_comm_strict.groupby("Commodity"):
    for col in (EXPORT_COL, IMPORT_COL):
        res = dl_fold_metrics_commodity_series(g, col, lookback=L, horizon=H)
        res["Commodity"] = commodity
        dl_cv_results_comm.append(res)

dl_cv_results_comm = (pd.concat(dl_cv_results_comm, ignore_index=True)
                      if dl_cv_results_comm else pd.DataFrame())

print("CNN-LSTM CV (clean_comm_strict by Commodity):", dl_cv_results_comm.shape)
display(dl_cv_results_comm.head())

CNN-LSTM CV (clean_comm_strict by Commodity): (5400, 6)


,Year,sMAPE,MASE,used,Series,Commodity
0,2020,0.199191,1.761976,CNN-LSTM,export_value,111 Agricultural Products
1,2021,0.286916,2.429716,CNN-LSTM,export_value,111 Agricultural Products
2,2022,0.351606,2.671061,CNN-LSTM,export_value,111 Agricultural Products
3,2023,0.245441,1.645570,CNN-LSTM,export_value,111 Agricultural Products
4,2020,0.065625,1.528817,CNN-LSTM,import_value,111 Agricultural Products


Now compare the two models using the weights mentioned before:

In [18]:
def weighted_summary(cv_df, id_cols=["Commodity","Series"]):
    df = cv_df.dropna(subset=["sMAPE","MASE"]).copy()
    df["w"] = df["Year"].map(WEIGHTS).astype(float)
    out = (df.groupby(id_cols)
             .apply(lambda d: pd.Series({
                 "w_sMAPE": np.average(d["sMAPE"], weights=d["w"]),
                 "w_MASE":  np.average(d["MASE"],  weights=d["w"]),
             }))
             .reset_index())
    return out

# Weighted summaries (ETS & CNN)
ets_wsummary = weighted_summary(ets_cv_results_comm)
cnn_wsummary = weighted_summary(dl_cv_results_comm)


/var/folders/6z/9dzyg6s964d4mtqp3fy4sxtr0000gn/T/ipykernel_74944/3771842428.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({
/var/folders/6z/9dzyg6s964d4mtqp3fy4sxtr0000gn/T/ipykernel_74944/3771842428.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda d: pd.Series({


In [20]:
# Compact accuracy summary (clean countries)
summary_acc = (
    ets_wsummary.rename(columns={"w_sMAPE":"ETS_w_sMAPE","w_MASE":"ETS_w_MASE"})
    .merge(cnn_wsummary.rename(columns={"w_sMAPE":"CNN_w_sMAPE","w_MASE":"CNN_w_MASE"}),
           on=["Commodity","Series"], how="inner")
)
summary_acc["sMAPE_gain_%"] = 100.0 * (summary_acc["ETS_w_sMAPE"] - summary_acc["CNN_w_sMAPE"]) / summary_acc["ETS_w_sMAPE"]
acc_report = pd.DataFrame({
    "Pairs": [len(summary_acc)],
    "Mean sMAPE gain vs ETS (%)": [summary_acc["sMAPE_gain_%"].mean()],
    "Share MASE<1 (CNN)": [(summary_acc["CNN_w_MASE"] < 1.0).mean()],
    "Share MASE<1 (ETS)": [(summary_acc["ETS_w_MASE"] < 1.0).mean()],
})
acc_report


,Pairs,Mean sMAPE gain vs ETS (%),Share MASE<1 (CNN),Share MASE<1 (ETS)
0,1350,98.366512,0.134815,0.351111


Interpretation: 

1. On average, CNN-LSTM’s recency-weighted sMAPE is ~98.4% lower than ETS’s
2. 13.5% of series had CNN-LSTM beating seasonal-naive (MASE<1)
3. 35.1% of series had ETS beating seasonal-naive

In [22]:
# Merge weighted results
WEIGHTS = {2020: 0.1, 2021: 0.2, 2022: 0.3, 2023: 0.4}


cmp_w = (
    ets_wsummary.rename(columns={"w_sMAPE":"ETS_w_sMAPE","w_MASE":"ETS_w_MASE"})
    .merge(
        cnn_wsummary.rename(columns={"w_sMAPE":"CNN_w_sMAPE","w_MASE":"CNN_w_MASE"}),
        on=["Commodity","Series"],
        how="inner"
    )
)

# Winner by weighted sMAPE (tie-breaker: weighted MASE; final tie -> ETS)
def pick_winner_weighted(row):
    if pd.isna(row["ETS_w_sMAPE"]) and pd.isna(row["CNN_w_sMAPE"]): 
        return "NA"
    if pd.isna(row["ETS_w_sMAPE"]): 
        return "CNN"
    if pd.isna(row["CNN_w_sMAPE"]): 
        return "ETS"
    if row["ETS_w_sMAPE"] < row["CNN_w_sMAPE"] - 1e-12:
        return "ETS"
    if row["CNN_w_sMAPE"] < row["ETS_w_sMAPE"] - 1e-12:
        return "CNN"
    # tie on sMAPE -> check MASE
    if pd.isna(row["ETS_w_MASE"]) and pd.isna(row["CNN_w_MASE"]): 
        return "ETS"
    if pd.isna(row["ETS_w_MASE"]): 
        return "CNN"
    if pd.isna(row["CNN_w_MASE"]): 
        return "ETS"
    if row["ETS_w_MASE"] <= row["CNN_w_MASE"]:
        return "ETS"
    return "CNN"

cmp_w["Winner"] = cmp_w.apply(pick_winner_weighted, axis=1)

winner_counts_weighted = (cmp_w.groupby(["Series","Winner"])
                            .size()
                            .reset_index(name="Count")
                            .sort_values(["Series","Winner"]))


# Per-commodity overall pick (Exports & Imports): winner by series count; tie -> lower avg *weighted* sMAPE
tmp = (cmp_w
       .assign(ETS_win=(cmp_w["Winner"]=="ETS").astype(int),
               CNN_win=(cmp_w["Winner"]=="CNN").astype(int))
       .groupby("Commodity", as_index=False)
       .agg(ETS_series_wins=("ETS_win","sum"),
            CNN_series_wins=("CNN_win","sum"),
            ETS_avg_w_sMAPE=("ETS_w_sMAPE","mean"),
            CNN_avg_w_sMAPE=("CNN_w_sMAPE","mean")))

def pick_overall(row):
    if row["ETS_series_wins"] > row["CNN_series_wins"]:
        return "ETS"
    if row["CNN_series_wins"] > row["ETS_series_wins"]:
        return "CNN"
    # tie by series wins -> lower avg weighted sMAPE
    return "ETS" if row["ETS_avg_w_sMAPE"] <= row["CNN_avg_w_sMAPE"] else "CNN"

country_overall = tmp.copy()
country_overall["Overall_Winner"] = country_overall.apply(pick_overall, axis=1)


# Outputs:
# - cmp_w: row-wise comparison per Country x Series with the winner
# - winner_counts_weighted: how many wins per series per method
# - country_overall: single overall winner per sountry (considering both series)

# quick looks:
# cmp.head()
# winner_counts
# country_overall.head()


In [23]:
country_overall.head()

,Commodity,ETS_series_wins,CNN_series_wins,ETS_avg_w_sMAPE,CNN_avg_w_sMAPE,Overall_Winner
0,111 Agricultural Products,0,2,12.156320,0.246536,CNN
1,1111 Oilseeds & Grains,0,2,23.319795,0.333582,CNN
2,11111 Soybeans,0,2,53.983476,0.543107,CNN
3,111110 Soybeans,0,2,53.983476,0.543107,CNN
4,11112 Oilseeds (except Soybean),0,2,70.482251,0.290799,CNN


In [25]:
winner_counts_weighted

,Series,Winner,Count
0,export_value,CNN,675
1,import_value,CNN,675


On clean commodities, CNN-LSTM beats ETS by recency-weighted sMAPE across both flows (winner count: 675/675). ETS used additive trend + additive seasonality (m=12, optional damping) while CNN-LSTM uses shallow Conv1d + LSTM with early stop.

Since CNN is the clear winner, we will use cnn-lstm to train on 2015-2023 and predict for 2024 but only for the clean_comm_strict dataset first

In [28]:
clean_full_commodities = sorted(clean_comm_strict["Commodity"].unique())

pred_rows = []  # (Commodity, SeriesTag, Month, PredUSD)

for commodity, g in clean_comm_strict.groupby("Commodity"):
    if commodity not in set(clean_full_commodities):
        continue
    for series_tag, col in [("export", EXPORT_COL), ("import", IMPORT_COL)]:
        # Build monthly series and restrict final fit to 2015–2023
        s_all = to_ts(g, col)  # must return MS frequency using Year + month_num
        s = s_all[(s_all.index.year >= 2015) & (s_all.index.year <= 2023)]

        y = s.values.astype(float)
        y_log = np.log1p(np.clip(y, a_min=0, a_max=None))

        if len(y_log) < L + H or len(y) < 12:
            # seasonal-naive fallback (or repeat last value if <12 months)
            if len(y) >= 12:
                pred_usd = y[-12:].astype(float)
            else:
                last_val = float(y[-1]) if len(y) else 0.0
                pred_usd = np.array([last_val]*12, dtype=float)
        else:
            # standardize on all 2015–2023 (final fit period)
            mu, sd = float(y_log.mean()), float(y_log.std(ddof=0))
            if sd == 0: sd = 1.0
            y_std = (y_log - mu) / sd

            # supervised windows across full train span
            X, Y = make_supervised(y_std, lookback=L, horizon=H)
            n = X.shape[0]
            split = max(1, int(np.floor(0.9*n)))  # last 10% as val
            X_tr, Y_tr = X[:split], Y[:split]
            X_va, Y_va = (X[split:], Y[split:]) if split < n else (X[:1], Y[:1])

            train_loader = DataLoader(WindowDataset(X_tr, Y_tr), batch_size=64, shuffle=True)
            val_loader = DataLoader(WindowDataset(X_va, Y_va), batch_size=64, shuffle=False)

            torch.manual_seed(42)
            model = CNNLSTMForecaster(L=L, H=H, in_feats=1,
                                      conv_filters=32, k=5, lstm_units=64, dropout=0.2).to(DEVICE)
            model = train_earlystop(model, train_loader, val_loader, epochs=200, lr=1e-3, patience=20)

            # forecast next 12 from the last lookback window ending Dec-2023
            last_win_std = ((y_log[-L:] - mu) / sd).reshape(1, L, 1)
            with torch.no_grad():
                pred_std = model(torch.from_numpy(last_win_std).float().to(DEVICE)).cpu().numpy().ravel()
            pred_log = pred_std * sd + mu
            pred_usd = np.expm1(pred_log).clip(min=0)

        for m_idx, m in enumerate(range(1, 13)):
            pred_rows.append((commodity, series_tag, m, float(pred_usd[m_idx])))

# Build tidy prediction frame for mapping
pred_df = pd.DataFrame(pred_rows, columns=["Commodity","Series","Month","Pred"])

# Write into submission (commodities, Year 2024 only)
mask = submission["Commodity"].isin(clean_full_commodities) & (submission["Year"] == 2024)

exp_map = pred_df.loc[pred_df["Series"]=="export"].set_index(["Commodity","Month"])["Pred"].to_dict()
imp_map = pred_df.loc[pred_df["Series"]=="import"].set_index(["Commodity","Month"])["Pred"].to_dict()

submission.loc[mask, "Pred_Export_USD"] = submission.loc[mask, ["Commodity","Month"]].apply(
    lambda r: exp_map.get((r["Commodity"], int(r["Month"]))), axis=1
)
submission.loc[mask, "Pred_Import_USD"] = submission.loc[mask, ["Commodity","Month"]].apply(
    lambda r: imp_map.get((r["Commodity"], int(r["Month"]))), axis=1
)

print("Filled submission rows for 2024:", mask.sum())

Filled submission rows for 2024: 8100


## Modeling for commodities whose export or import were ALL 0 from 2015-01 until 2023-12

In [33]:
# Ensure numeric
gaps_comm["export_value"] = pd.to_numeric(gaps_comm["export_value"], errors="coerce")
gaps_comm["import_value"] = pd.to_numeric(gaps_comm["import_value"], errors="coerce")

# Restrict to the training window
dfz = gaps_comm[(gaps_comm["Year"] >= 2015) & (gaps_comm["Year"] <= 2023)].copy()

# True iff there is NO strictly positive observation across the whole window
def _all_na_or_zero(s: pd.Series) -> bool:
    return ~(s.fillna(0) > 0).any()

# Flags per Commodity
flags = (dfz.groupby("Commodity", as_index=False)
           .agg(export_all_na_zero=("export_value", _all_na_or_zero),
                import_all_na_zero=("import_value", _all_na_or_zero)))

# Sets to zero
exp_zero_set = set(flags.loc[flags["export_all_na_zero"], "Commodity"])
imp_zero_set = set(flags.loc[flags["import_all_na_zero"], "Commodity"])

# Update submission (Year == 2024 only)
mask_2024 = submission["Year"].eq(2024)

n_before_exp = submission.loc[mask_2024 & submission["Commodity"].isin(exp_zero_set), "Pred_Export_USD"].notna().sum()
n_before_imp = submission.loc[mask_2024 & submission["Commodity"].isin(imp_zero_set), "Pred_Import_USD"].notna().sum()

submission.loc[mask_2024 & submission["Commodity"].isin(exp_zero_set), "Pred_Export_USD"] = 0.0
submission.loc[mask_2024 & submission["Commodity"].isin(imp_zero_set), "Pred_Import_USD"] = 0.0

rows_exp = (mask_2024 & submission["Commodity"].isin(exp_zero_set)).sum()
rows_imp = (mask_2024 & submission["Commodity"].isin(imp_zero_set)).sum()
print(f"[Zero-flow override] Export-zero: rows touched={rows_exp} (prev non-NaNs={n_before_exp})")
print(f"[Zero-flow override] Import-zero: rows touched={rows_imp} (prev non-NaNs={n_before_imp})")

# quick sanity checking
print("Commodities flagged export_zero:", len(exp_zero_set))
print("Commodities flagged import_zero:", len(imp_zero_set))

[Zero-flow override] Export-zero: rows touched=84 (prev non-NaNs=0)
[Zero-flow override] Import-zero: rows touched=12 (prev non-NaNs=0)
Commodities flagged export_zero: 7
Commodities flagged import_zero: 1


## Modeling for commodities that are problematic with gaps in their import or export

Now we deal with the commodities that have gaps in either import or export but are not ALL zeros.
To deal with this, we compute an effective zero fraction for each (Commodity, flow) over the entire 2015–2023 monthly grid (108 months). 

Then, based on the results, we will predict using two models:
1. ETS (log1p, seasonal=12) if zero_eff_frac_total < 0.5 as it has enough signal to learn trend/seasonality.
2. TSB if zero_eff_frac_total ≥ 0.5 as it has intermittent series TSB models occurrence and size

In [35]:
gc = gaps_comm.copy()
gc["date"] = pd.to_datetime(gc["date"], errors="coerce")

# Long format with flows
long = (gc.melt(id_vars=["Commodity","date"],
                value_vars=["export_value","import_value"],
                var_name="flow", value_name="value")
          .assign(flow=lambda d: np.where(d["flow"].eq("export_value"), "export", "import"))
          .dropna(subset=["date"])
          .sort_values(["Commodity","flow","date"])
       )

# Full monthly index (2015-01 - 2023-12)
full_idx = pd.date_range("2015-01-01", "2023-12-01", freq="MS")
N_FULL = len(full_idx)  # 108

def zero_eff_frac_total(sdf: pd.DataFrame) -> float:
    """Fraction of months that are NA or 0 over the *full* 2015–2023 grid."""
    s = (sdf.set_index("date")[["value"]]
           .reindex(full_idx) # add missing months
           .squeeze("columns")) # Series of length 108
    # Count NA or 0 as zeros (intermittency/no activity)
    z = s.isna() | (s.fillna(0.0) == 0.0)
    return float(z.sum()) / N_FULL

# Compute per (Commodity, flow)
routes_pair = (long.groupby(["Commodity","flow"], as_index=False)
                    .apply(lambda g: pd.Series({
                        "zero_eff_frac_total": zero_eff_frac_total(g[["date","value"]]),
                        "n_present": g["value"].notna().sum(),
                        "first_obs": g["date"].min(),
                        "last_obs":  g["date"].max()
                    }))
                    .reset_index(drop=True))

# Flag “effectively all-zero” (handled elsewhere) vs routing threshold
routes_pair["all_zero_eff"] = routes_pair["zero_eff_frac_total"].ge(0.999)  # ~all months NA/0
routes_pair["route"] = np.where(
    routes_pair["zero_eff_frac_total"] < 0.50, "ETS", "TSB"
)

# Summary prints
print("Total (Commodity, flow) in gaps_comm:", routes_pair.shape[0])
print("Effectively all-zero flows (≈108/108 NA or 0):", int(routes_pair["all_zero_eff"].sum()))
print("Routing counts (excluding all-zero):")
print(routes_pair.loc[~routes_pair["all_zero_eff"], "route"].value_counts())

# Quick sanity: quantiles
print("\nzero_eff_frac_total quantiles (excluding all-zero):")
print(routes_pair.loc[~routes_pair["all_zero_eff"], "zero_eff_frac_total"]
                   .quantile([0,0.25,0.5,0.75,0.9,0.95,1.0]).round(3))

# Peek a few examples from each route
print("\nExamples — ETS route (<0.5):")
display(routes_pair.query("~all_zero_eff & route=='ETS'")
                   .sort_values("zero_eff_frac_total").head(10))
print("\nExamples — TSB route (>=0.5):")
display(routes_pair.query("~all_zero_eff & route=='TSB'")
                   .sort_values("zero_eff_frac_total", ascending=False).head(10))

# Optional: commodity-level view (which flow goes where)
routes_commodity = (routes_pair.pivot(index="Commodity", columns="flow", values="route")
                               .rename_axis(columns=None)
                               .reset_index()
                               .rename(columns={"export":"export_route","import":"import_route"}))

print("\nCommodity-level routing (first 10):")
display(routes_commodity.head(10))


Total (Commodity, flow) in gaps_comm: 108
Effectively all-zero flows (≈108/108 NA or 0): 8
Routing counts (excluding all-zero):
route
ETS    66
TSB    34
Name: count, dtype: int64

zero_eff_frac_total quantiles (excluding all-zero):
0.00    0.000
0.25    0.028
0.50    0.333
0.75    0.556
0.90    0.728
0.95    0.778
1.00    0.926
Name: zero_eff_frac_total, dtype: float64

Examples — ETS route (<0.5):


/var/folders/6z/9dzyg6s964d4mtqp3fy4sxtr0000gn/T/ipykernel_74944/372569506.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


,Commodity,flow,zero_eff_frac_total,n_present,first_obs,last_obs,all_zero_eff,route
0,11116 Rice,export,0.0,108,2015-01-01,2023-12-01,False,ETS
102,"33641X Civilian Aircraft, Engines, Equipment, ...",export,0.0,108,2015-01-01,2023-12-01,False,ETS
100,336414 Guided Missiles & Space Vehicles,export,0.0,108,2015-01-01,2023-12-01,False,ETS
83,331529 Other Nonferrous Metal (except Die-casts),import,0.0,108,2015-01-01,2023-12-01,False,ETS
76,"327320 Wet, Nonrefractory Mortars & Concretes",export,0.0,108,2015-01-01,2023-12-01,False,ETS
74,"32732 Wet, Nonrefractory Mortars & Concretes",export,0.0,108,2015-01-01,2023-12-01,False,ETS
73,324191 Petroleum Lubricating Oil And Grease,import,0.0,108,2015-01-01,2023-12-01,False,ETS
63,311612 Processed Meat,import,0.0,108,2015-01-01,2023-12-01,False,ETS
52,"21223 Copper, Nickel, Lead & Zinc",export,0.0,108,2015-01-01,2023-12-01,False,ETS
44,21222 Gold Ores & Silver Ores,export,0.0,108,2015-01-01,2023-12-01,False,ETS



Examples — TSB route (>=0.5):


,Commodity,flow,zero_eff_frac_total,n_present,first_obs,last_obs,all_zero_eff,route
47,212220 Gold Ore And Silver Ore Mining,import,0.925926,8,2023-01-01,2023-12-01,False,TSB
57,212231 Lead Ores & Zinc Ores,import,0.907407,10,2015-01-01,2018-12-01,False,TSB
46,212220 Gold Ore And Silver Ore Mining,export,0.888889,12,2023-01-01,2023-12-01,False,TSB
51,212222 Silver Ores,import,0.888889,12,2015-01-01,2022-12-01,False,TSB
106,339995 Burial Caskets,export,0.777778,24,2015-01-01,2023-12-01,False,TSB
62,311612 Processed Meat,export,0.777778,24,2015-01-01,2023-12-01,False,TSB
72,324191 Petroleum Lubricating Oil And Grease,export,0.777778,24,2015-01-01,2023-12-01,False,TSB
27,115114 Crops Processed Post Harvest,import,0.777778,24,2022-01-01,2023-12-01,False,TSB
25,11511 Crops & Products Supporting Crop Production,import,0.777778,24,2022-01-01,2023-12-01,False,TSB
23,1151 Crops & Products Supporting Crop Production,import,0.777778,24,2022-01-01,2023-12-01,False,TSB



Commodity-level routing (first 10):


,Commodity,export_route,import_route
0,11116 Rice,ETS,TSB
1,111160 Rice,ETS,TSB
2,111419 Other Food Crops Grown Under Cover,TSB,TSB
3,11192 Cotton,ETS,ETS
4,111920 Cotton,ETS,ETS
5,11193 Sugarcane,ETS,ETS
6,111930 Sugarcane,ETS,ETS
7,111991 Sugar Beets,ETS,ETS
8,111992 Peanuts,ETS,ETS
9,11242 Goats & Other Fine Animal Hair,ETS,ETS


Now we will model accordingly:

In [37]:
gc = gaps_comm.copy()
gc["date"] = pd.to_datetime(gc["date"], errors="coerce")
for col in ["export_value","import_value"]:
    if col in gc.columns:
        gc[col] = pd.to_numeric(gc[col], errors="coerce")

# Long table (date,value,flow)
long = (gc.melt(id_vars=["Commodity","date"],
                value_vars=["export_value","import_value"],
                var_name="flow", value_name="value")
          .assign(flow=lambda d: np.where(d["flow"].eq("export_value"), "export", "import"))
          .dropna(subset=["date"])
          .sort_values(["Commodity","flow","date"])
       )

# Keep only non–all-zero flows from gaps_comm
routes_use = routes_pair.loc[~routes_pair["all_zero_eff"], ["Commodity","flow","route"]].copy()

FULL_IDX = pd.date_range("2015-01-01", "2023-12-01", freq="MS")

def complete_zero_fill(sdf: pd.DataFrame) -> pd.Series:
    """Complete to full 2015-01..2023-12 and treat missing months as true zeros."""
    s = (sdf.set_index("date")[["value"]]
           .reindex(FULL_IDX)
           .squeeze("columns")
           .fillna(0.0))
    return s.astype(float)

def ets_monthly_2024_from_series(s: pd.Series) -> np.ndarray:
    """ETS on log1p(s), seasonal=12, damped; fallback seasonal naive if failure."""
    y_log = np.log1p(s.values)
    try:
        mdl = ExponentialSmoothing(
            y_log, trend="add", seasonal="add", damped_trend=True,
            seasonal_periods=12, initialization_method="estimated"
        ).fit(optimized=True, use_brute=False)
        f = mdl.forecast(12) # log scale
        y = np.expm1(f)
    except Exception:
        last12 = s.values[-12:] if len(s) >= 12 else np.zeros(12, dtype=float)
        y = last12 if last12.size == 12 else np.zeros(12, dtype=float)
    return np.maximum(y, 0.0)

def tsb_forecast_12(x: np.ndarray, alpha=0.3, beta=0.3) -> np.ndarray:
    x = np.asarray(x, dtype=float)
    occ = (x > 0).astype(float)
    size = np.where(x > 0, x, 0.0)

    # Initialize: if no positive obs, predict zeros
    if occ.sum() == 0:
        return np.zeros(12, dtype=float)

    p = occ[0]
    # initial size as mean of positive sizes
    z_init = size[size > 0].mean() if (size > 0).any() else 0.0
    z = z_init

    for xi, oi in zip(x, occ):
        p = alpha * oi + (1 - alpha) * p
        if oi == 1.0:
            z = beta * xi + (1 - beta) * z
        # if oi==0, z stays unchanged

    f = p * z
    return np.full(12, max(f, 0.0), dtype=float)

# Build forecasts per (Commodity, flow) according to which model

rows = []
for _, r in routes_use.iterrows():
    c, f, route = r["Commodity"], r["flow"], r["route"]
    sdf = long[(long["Commodity"]==c) & (long["flow"]==f)][["date","value"]]
    if sdf.empty:
        continue
    s = complete_zero_fill(sdf)  # monthly 2015-01-2023-12, zeros where NA
    if route == "ETS":
        y = ets_monthly_2024_from_series(s)
    else:  # TSB
        y = tsb_forecast_12(s.values, alpha=0.3, beta=0.3)
    dates_2024 = pd.date_range("2024-01-01", periods=12, freq="MS")
    rows += [(c, f, d, float(v)) for d, v in zip(dates_2024, y)]

pred_gaps_all = pd.DataFrame(rows, columns=["Commodity","flow","date","pred_usd"])
pred_gaps_all["Year"]  = 2024
pred_gaps_all["Month"] = pred_gaps_all["date"].dt.month
pred_gaps_all = pred_gaps_all.drop(columns=["date"])

# Split to export / import columns for merging into submission
exp_fill = (pred_gaps_all[pred_gaps_all["flow"]=="export"]
            .rename(columns={"pred_usd":"_tmp_export"}))[["Commodity","Year","Month","_tmp_export"]]
imp_fill = (pred_gaps_all[pred_gaps_all["flow"]=="import"]
            .rename(columns={"pred_usd":"_tmp_import"}))[["Commodity","Year","Month","_tmp_import"]]

# Fill current submission

# Ensure dtypes
submission["Year"] = submission["Year"].astype(int)
submission["Month"] = pd.to_numeric(submission["Month"], errors="coerce").astype(int)

# Counts before
n_before = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()

# Merge export fills, update, drop temp
submission = submission.merge(exp_fill, on=["Commodity","Year","Month"], how="left")
mask_e = submission["Pred_Export_USD"].isna() & submission["_tmp_export"].notna()
submission.loc[mask_e, "Pred_Export_USD"] = submission.loc[mask_e, "_tmp_export"]
submission.drop(columns=["_tmp_export"], inplace=True)

# Merge import fills, update, drop temp
submission = submission.merge(imp_fill, on=["Commodity","Year","Month"], how="left")
mask_i = submission["Pred_Import_USD"].isna() & submission["_tmp_import"].notna()
submission.loc[mask_i, "Pred_Import_USD"] = submission.loc[mask_i, "_tmp_import"]
submission.drop(columns=["_tmp_import"], inplace=True)

# Report
n_after = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()
print("Filled via GAPS routing (ETS & TSB) — export cells:",
      int(n_before["Pred_Export_USD"] - n_after["Pred_Export_USD"]),
      "| import cells:", int(n_before["Pred_Import_USD"] - n_after["Pred_Import_USD"]))
print("Remaining NA — export:", int(n_after["Pred_Export_USD"]),
      "| import:", int(n_after["Pred_Import_USD"]))


Filled via GAPS routing (ETS & TSB) — export cells: 0 | import cells: 0
Remaining NA — export: 1008 | import: 1008


## Modeling for commodities that starts late (has data only from 2021 - 2023 / 2022 - 2023)

In [39]:
# Ensure datetime
starts_late['date'] = pd.to_datetime(starts_late['date'])

# Long format and count distinct years with non-NA values
panel = (starts_late.melt(id_vars=['Commodity','date'],
                          value_vars=['export_value','import_value'],
                          var_name='flow', value_name='value')
                   .assign(flow=lambda d: np.where(d['flow'].eq('export_value'), 'export', 'import')))

panel['Year'] = panel['date'].dt.year

# Number of years per (Commodity, flow)
years_per_series = (panel.dropna(subset=['value'])
                           .groupby(['Commodity','flow'], as_index=False)['Year']
                           .nunique()
                           .rename(columns={'Year':'n_years'}))

# Distribution: how many series have 1, 2, 3, ... years
print(years_per_series['n_years'].value_counts().sort_index())

# Peek the first few rows
years_per_series.head(20)


n_years
1    46
2    30
Name: count, dtype: int64


,Commodity,flow,n_years
0,1121X Cattle,export,2
1,1121X Cattle,import,2
2,1121XX Cattle,export,2
3,1121XX Cattle,import,2
4,115 Products Supporting Agriculture And Forestry,export,2
5,115 Products Supporting Agriculture And Forestry,import,2
6,1152 Support Activities For Animal Production,export,2
7,1152 Support Activities For Animal Production,import,2
8,11521 Support Activities For Animal Production,export,2
9,11521 Support Activities For Animal Production,import,2


#### Lets first view the commodities whose export are either only from 2022-2023 or just 2023

In [40]:
sl = starts_late.copy()
sl['date'] = pd.to_datetime(sl['date'])

panel = (sl.melt(id_vars=['Commodity','date'],
                 value_vars=['export_value','import_value'],
                 var_name='flow', value_name='value')
           .assign(flow=lambda d: np.where(d['flow'].eq('export_value'), 'export', 'import'),
                   Year=lambda d: d['date'].dt.year))

years_per_series = (panel.dropna(subset=['value'])
                           .groupby(['Commodity','flow'], as_index=False)['Year']
                           .nunique()
                           .rename(columns={'Year':'n_years'}))

one_year  = years_per_series.query("n_years == 1").sort_values(['flow','Commodity'])
two_years = years_per_series.query("n_years == 2").sort_values(['flow','Commodity'])

print("=== 1-year series (export) ===")
display(one_year[one_year['flow']=='export'][['Commodity']])
print("=== 1-year series (import) ===")
display(one_year[one_year['flow']=='import'][['Commodity']])

print("=== 2-year series (export) ===")
display(two_years[two_years['flow']=='export'][['Commodity']])
print("=== 2-year series (import) ===")
display(two_years[two_years['flow']=='import'][['Commodity']])



=== 1-year series (export) ===


,Commodity
12,212114 Surface Coal Mining
14,212115 Underground Coal Mining
16,212290 Other Metal Ore Mining
18,"212323 Kaolin, Clay, Ceramic And Refrac Minera..."
20,212390 Other Nonmetallic Mineral Mining And Qu...
22,31512 Apparel Knitting Mills
24,315120 Apparel Knitting Mills
26,31525 Cut And Sew Apparel Manufact (except Con...
28,315250 Cut And Sew Apparel Manufact (except Co...
30,316990 Other Leather And Allied Product Manufa...


=== 1-year series (import) ===


,Commodity
13,212114 Surface Coal Mining
15,212115 Underground Coal Mining
17,212290 Other Metal Ore Mining
19,"212323 Kaolin, Clay, Ceramic And Refrac Minera..."
21,212390 Other Nonmetallic Mineral Mining And Qu...
23,31512 Apparel Knitting Mills
25,315120 Apparel Knitting Mills
27,31525 Cut And Sew Apparel Manufact (except Con...
29,315250 Cut And Sew Apparel Manufact (except Co...
31,316990 Other Leather And Allied Product Manufa...


=== 2-year series (export) ===


,Commodity
0,1121X Cattle
2,1121XX Cattle
4,115 Products Supporting Agriculture And Forestry
6,1152 Support Activities For Animal Production
8,11521 Support Activities For Animal Production
10,115210 Support Activities For Animal Production
34,"321912 Cut Stock, Resawing Lumber, And Planing"
38,32614 Polystyrene Foam Product Manufacturing
40,326140 Polystyrene Foam Products
42,32615 Urethane And Oth Foam Products (exc Poly...


=== 2-year series (import) ===


,Commodity
1,1121X Cattle
3,1121XX Cattle
5,115 Products Supporting Agriculture And Forestry
7,1152 Support Activities For Animal Production
9,11521 Support Activities For Animal Production
11,115210 Support Activities For Animal Production
35,"321912 Cut Stock, Resawing Lumber, And Planing"
39,32614 Polystyrene Foam Product Manufacturing
41,326140 Polystyrene Foam Products
43,32615 Urethane And Oth Foam Products (exc Poly...


##### Predicting for the commodities with export and import only in 2022 and 2023

In [43]:
keys = two_years[['Commodity','flow']].copy()

# helper: seasonal mean-of-month fallback 
def seasonal_mean_of_month(df):
    d = df.copy()
    d['m'] = d['date'].dt.month
    avg = d.dropna(subset=['value']).groupby('m')['value'].mean()
    months = np.arange(1,13)
    vals = np.array([avg.get(m, d['value'].iloc[-1]) for m in months], dtype=float)
    dates = pd.date_range('2024-01-01', periods=12, freq='MS')
    return dates, vals

# helper: ETS on log1p with seasonality -
def ets_2024(df):
    s = df.sort_values('date')['value'].astype(float)
    y = np.log1p(s.clip(lower=0))
    if y.notna().sum() < 18:
        return seasonal_mean_of_month(df)
    try:
        mdl = ExponentialSmoothing(
            y.values, trend='add', seasonal='add', damped_trend=True,
            seasonal_periods=12, initialization_method='estimated'
        ).fit(optimized=True, use_brute=False)
        f = np.expm1(mdl.forecast(12)).clip(min=0.0)
        dates = pd.date_range('2024-01-01', periods=12, freq='MS')
        return dates, f
    except Exception:
        return seasonal_mean_of_month(df)

# build predictions 
rows = []
g = panel  # your long df with [Commodity, flow, date, value, Year]
for _, r in keys.iterrows():
    c, f = r['Commodity'], r['flow']
    df = g[(g['Commodity']==c) & (g['flow']==f)].dropna(subset=['value'])
    if df.empty: 
        continue
    dts, vals = ets_2024(df)
    rows += [(c, f, d, float(v)) for d, v in zip(dts, vals)]

pred_2yr = pd.DataFrame(rows, columns=['Commodity','flow','date','pred_usd'])
print("2-year series monthly 2024 preds:", pred_2yr.shape)

# Fill submission 

# Build monthly maps from pred_2yr
tidy = (pred_2yr
        .assign(Year=2024, Month=lambda d: d["date"].dt.month)
        [["Commodity","Year","Month","flow","pred_usd"]])

exp2 = (tidy[tidy["flow"]=="export"]
        [["Commodity","Year","Month","pred_usd"]]
        .rename(columns={"pred_usd":"Pred_Export_USD"}))

imp2 = (tidy[tidy["flow"]=="import"]
        [["Commodity","Year","Month","pred_usd"]]
        .rename(columns={"pred_usd":"Pred_Import_USD"}))

# Ensure dtypes on submission
submission["Year"]  = submission["Year"].astype(int)
submission["Month"] = pd.to_numeric(submission["Month"], errors="coerce").astype(int)

# Count missing before
n_before = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()

# Merge export fills and apply only to NaNs
submission = submission.merge(exp2, on=["Commodity","Year","Month"], how="left", suffixes=("", "_2Y"))
mask_e = submission["Pred_Export_USD"].isna() & submission["Pred_Export_USD_2Y"].notna()
submission.loc[mask_e, "Pred_Export_USD"] = submission.loc[mask_e, "Pred_Export_USD_2Y"]
submission.drop(columns=["Pred_Export_USD_2Y"], inplace=True)

# Merge import fills and apply only to NaNs
submission = submission.merge(imp2, on=["Commodity","Year","Month"], how="left", suffixes=("", "_2Y"))
mask_i = submission["Pred_Import_USD"].isna() & submission["Pred_Import_USD_2Y"].notna()
submission.loc[mask_i, "Pred_Import_USD"] = submission.loc[mask_i, "Pred_Import_USD_2Y"]
submission.drop(columns=["Pred_Import_USD_2Y"], inplace=True)

# Report
filled_count = int(mask_e.sum() + mask_i.sum())
n_after = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()
print(f"Filled cells (starts_late 2-year): {filled_count}")
print("Remaining NA — export:", int(n_after["Pred_Export_USD"]),
      "| import:", int(n_after["Pred_Import_USD"]))


2-year series monthly 2024 preds: (360, 4)
Filled cells (starts_late 2-year): 360
Remaining NA — export: 828 | import: 828


##### Predicting for the commodities with export and import only in 2023

In [44]:
keys_1y = one_year[['Commodity','flow']].copy()

def seasonal_naive_from_last_year(df):
    d = df.copy()
    d['Year'] = d['date'].dt.year
    d['Month'] = d['date'].dt.month
    last_year = d['Year'].max()
    d_last = d[d['Year'] == last_year]

    # month -> last observed value in that month of last_year (if multiple rows)
    last_vals = (d_last.dropna(subset=['value'])
                       .sort_values('date')
                       .groupby('Month')['value']
                       .last())

    # month means across all history (fallback if that month missing in last_year)
    month_means = (d.dropna(subset=['value'])
                     .groupby('Month')['value']
                     .mean())

    # global last observed fallback
    global_last = d.dropna(subset=['value']).sort_values('date')['value'].iloc[-1] if d.dropna(subset=['value']).shape[0] else 0.0

    out = []
    for m in range(1, 13):
        v = last_vals.get(m, np.nan)
        if pd.isna(v):
            v = month_means.get(m, np.nan)
        if pd.isna(v):
            v = global_last
        out.append(float(v))
    dates = pd.date_range('2024-01-01', periods=12, freq='MS')
    return dates, np.array(out, dtype=float)

# Build 2024 predictions for 1-year series
rows = []
g = panel  # your long df
for _, r in keys_1y.iterrows():
    c, f = r['Commodity'], r['flow']
    df = g[(g['Commodity'] == c) & (g['flow'] == f)].dropna(subset=['value'])
    if df.empty:
        continue
    dts, vals = seasonal_naive_from_last_year(df)
    rows += [(c, f, d, float(v)) for d, v in zip(dts, vals)]

pred_1yr = pd.DataFrame(rows, columns=['Commodity','flow','date','pred_usd'])
print("1-year series monthly 2024 preds:", pred_1yr.shape)

# Fill submission 

# Build monthly maps from pred_1yr
tidy1 = (pred_1yr
         .assign(Year=2024, Month=lambda d: d["date"].dt.month)
         [["Commodity","Year","Month","flow","pred_usd"]])

exp1 = (tidy1[tidy1["flow"]=="export"]
        [["Commodity","Year","Month","pred_usd"]]
        .rename(columns={"pred_usd":"Pred_Export_USD"}))

imp1 = (tidy1[tidy1["flow"]=="import"]
        [["Commodity","Year","Month","pred_usd"]]
        .rename(columns={"pred_usd":"Pred_Import_USD"}))

# Ensure dtypes on submission
submission["Year"]= submission["Year"].astype(int)
submission["Month"] = pd.to_numeric(submission["Month"], errors="coerce").astype(int)

# Count missing before
n_before = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()

# Merge export 1Y preds and fill NaNs only
submission = submission.merge(exp1, on=["Commodity","Year","Month"], how="left", suffixes=("", "_1Y"))
mask_e = submission["Pred_Export_USD"].isna() & submission["Pred_Export_USD_1Y"].notna()
submission.loc[mask_e, "Pred_Export_USD"] = submission.loc[mask_e, "Pred_Export_USD_1Y"]
submission.drop(columns=["Pred_Export_USD_1Y"], inplace=True)

# Merge import 1Y preds and fill NaNs only
submission = submission.merge(imp1, on=["Commodity","Year","Month"], how="left", suffixes=("", "_1Y"))
mask_i = submission["Pred_Import_USD"].isna() & submission["Pred_Import_USD_1Y"].notna()
submission.loc[mask_i, "Pred_Import_USD"] = submission.loc[mask_i, "Pred_Import_USD_1Y"]
submission.drop(columns=["Pred_Import_USD_1Y"], inplace=True)

# Report
filled_count = int(mask_e.sum() + mask_i.sum())
n_after = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()
print(f"Filled cells (starts_late 1-year): {filled_count}")
print("Remaining NA — export:", int(n_after["Pred_Export_USD"]),
      "| import:", int(n_after["Pred_Import_USD"]))

1-year series monthly 2024 preds: (552, 4)
Filled cells (starts_late 1-year): 552
Remaining NA — export: 552 | import: 552


## Modeling for commodities that ends early (import/export is not up until 2023-12)

In [45]:
ee = ends_early.copy()
ee['date'] = pd.to_datetime(ee['date'])

# Long view to compute spans per (Commodity, flow)
panel = (ee.melt(id_vars=['Commodity','date'],
                 value_vars=['export_value','import_value'],
                 var_name='flow', value_name='value')
           .assign(flow=lambda d: np.where(d['flow'].eq('export_value'),'export','import'))
           .sort_values(['Commodity','flow','date']))

span = (panel.groupby(['Commodity','flow'], as_index=False)
             .agg(start_date=('date','min'),
                  end_date  =('date','max'),
                  n_months  =('date','nunique'),
                  n_non_na  =('value', lambda s: s.notna().sum())))
span['start_year'] = span['start_date'].dt.year
span['end_year']   = span['end_date'].dt.year
span['n_years']    = span['end_year'] - span['start_year'] + 1

# Option A: keep a tidy table per (Commodity, flow)
ends_early_spans = span.sort_values(['Commodity','flow'])
# display(ends_early_spans.head())

# Option B: add end_year (and other span cols) back to the wide ends_early table
wide_end_year = (span.pivot(index='Commodity', columns='flow', values='end_year')
                    .rename(columns={'export':'export_end_year','import':'import_end_year'})
                    .reset_index())

wide_n_years = (span.pivot(index='Commodity', columns='flow', values='n_years')
                   .rename(columns={'export':'export_n_years','import':'import_n_years'})
                   .reset_index())

ee_with_ends = (ee.merge(wide_end_year, on='Commodity', how='left')
                  .merge(wide_n_years, on='Commodity', how='left'))




In [46]:
ends_early_spans

,Commodity,flow,start_date,end_date,n_months,n_non_na,start_year,end_year,n_years
0,11211 Cattle,export,2015-01-01,2021-12-01,84,84,2015,2021,7
1,11211 Cattle,import,2015-01-01,2021-12-01,84,84,2015,2021,7
2,11211X Cattle,export,2015-01-01,2021-12-01,84,84,2015,2021,7
3,11211X Cattle,import,2015-01-01,2021-12-01,84,84,2015,2021,7
4,212111 Lignite,export,2015-01-01,2022-12-01,96,96,2015,2022,8
...,...,...,...,...,...,...,...,...,...
87,336112 Light Truck And Utility Vehicle,import,2015-01-01,2022-12-01,96,96,2015,2022,8
88,337124 Metal Household Furniture,export,2015-01-01,2022-12-01,96,96,2015,2022,8
89,337124 Metal Household Furniture,import,2015-01-01,2022-12-01,96,96,2015,2022,8
90,337125 Household Furniture (except Wood And Me...,export,2015-01-01,2022-12-01,96,96,2015,2022,8


In [47]:
# coerce numeric and dates
for df in (ends_early, clean_comm_strict):
    df["date"] = pd.to_datetime(df["date"])
    for col in ["export_value","import_value"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

# Long format with flow label
def to_long(df):
    out = (df.melt(id_vars=["Commodity","date"],
                   value_vars=["export_value","import_value"],
                   var_name="flow", value_name="value")
             .assign(flow=lambda d: np.where(d["flow"].eq("export_value"), "export", "import"),
                     Year=lambda d: d["date"].dt.year,
                     Month=lambda d: d["date"].dt.month)
             .sort_values(["Commodity","flow","date"]))
    return out

ee_panel = to_long(ends_early)
strict_panel = to_long(clean_comm_strict)

# where do the ends_early series end?
span_ee = (ee_panel.groupby(["Commodity","flow"], as_index=False)
                 .agg(start=("date","min"),
                      end=("date","max"),
                      n_months=("date","nunique"),
                      n_non_na=("value", lambda s: s.notna().sum())))

span_ee["start_year"] = span_ee["start"].dt.year
span_ee["last_year"]  = span_ee["end"].dt.year

print("ends_early — distribution of last_year:")
print(span_ee["last_year"].value_counts().sort_index())

# Keep donors that truly go through 2023-12 (robustness)
ends_strict = (strict_panel.groupby(["Commodity","flow"], as_index=False)
                           .agg(end=("date","max")))
strict_keys_full = ends_strict[ends_strict["end"] >= pd.Timestamp("2023-12-01")][["Commodity","flow"]]
print("Donor universe (strict, ends 2023-12):", len(strict_keys_full))


ends_early — distribution of last_year:
last_year
2021     6
2022    86
Name: count, dtype: int64
Donor universe (strict, ends 2023-12): 1350


### Modeling for commodities that end early (donor-based continuation)

We borrow seasonality + short-run movement from donor series that do reach 2023-12

How we rank donors:
- Compute target vs donor MoM log-diff correlation on their overlap (needs ≥6 months).
- Compare month-of-year seasonal profiles via cosine similarity (centered to remove level).
- Score = 0.7 × corr + 0.3 × cosine.

Candidate pools (progressively relaxed):
- same prefix + flow, 2) same flow (any prefix), 3) cosine-only fallback if corr too low.

Thresholds: keep donors with corr ≥ 0.30 (fallback 0.20). Take top-8 by score.

Outputs: donor_rank (top donors + diagnostics) and strict_keys_full (eligible donors ending at 2023-12).

In short, we find find close donors then rank by movement + seasonality then use best ones to extend 2024.

Find ranked donor list per target

In [48]:
# extract leading code/prefix from Commodity (example: "11211" or "33641X") 
def extract_prefix(name: str) -> str:
    if not isinstance(name, str):
        return ""
    m = re.match(r"^([0-9]+[A-ZX]*)", name.strip())
    return m.group(1) if m else ""

ee_panel = ee_panel.copy()
strict_panel = strict_panel.copy()
ee_panel["prefix"] = ee_panel["Commodity"].apply(extract_prefix)
strict_panel["prefix"] = strict_panel["Commodity"].apply(extract_prefix)

# Keep only donors that truly end in 2023-12
strict_panel = strict_panel.merge(strict_keys_full, on=["Commodity","flow"], how="inner")

# Build fast access series dictionaries by (Commodity, flow)
def series_dict(df):
    out = {}
    for (c,f), g in df.sort_values("date").groupby(["Commodity","flow"]):
        s = pd.Series(g["value"].astype(float).values, index=g["date"].values)
        out[(c,f)] = s
    return out

targets_series = series_dict(ee_panel)
donors_series  = series_dict(strict_panel)

# Month-of-year profile function
def month_profile(s: pd.Series):
    if s.empty:
        return np.full(12, np.nan)
    ts = pd.Series(s.values, index=pd.to_datetime(s.index))
    prof = ts.groupby(ts.index.month).mean()
    v = np.array([prof.get(m, np.nan) for m in range(1,13)], dtype=float)
    # center to compare shapes
    mu = np.nanmean(v)
    return (v - mu) if np.isfinite(mu) else v

def cosine_sim(a, b):
    a = np.asarray(a, float); b = np.asarray(b, float)
    mask = np.isfinite(a) & np.isfinite(b)
    if mask.sum() == 0: 
        return np.nan
    a = a[mask]; b = b[mask]
    na = np.linalg.norm(a); nb = np.linalg.norm(b)
    if na == 0 or nb == 0: 
        return np.nan
    return float(np.dot(a,b) / (na*nb))

def mom_logdiff(s: pd.Series):
    # log1p to stabilize; month-over-month diff
    x = np.log1p(np.clip(s.values, 0, None))
    d = np.diff(x)
    # align dates to the end of diff (second..last)
    idx = pd.to_datetime(s.index)[1:]
    return pd.Series(d, index=idx)

# Precompute donor seasonal profiles (full 2015–2023)
donor_profiles = {}
for key, s in donors_series.items():
    donor_profiles[key] = month_profile(s)

# parameters
TOP_K    = 8
MIN_CORR1 = 0.30
MIN_CORR2 = 0.20

def score_candidates(target_s, donor_keys, end_date):
    # target features on overlap
    s_t_overlap = target_s[target_s.index <= np.datetime64(end_date)]
    d_t = mom_logdiff(s_t_overlap)
    prof_t = month_profile(s_t_overlap)

    out = []
    for _, r in donor_keys.iterrows():
        key_d = (r["Commodity"], r["flow"])
        s_d_full = donors_series.get(key_d)
        if s_d_full is None or s_d_full.empty:
            continue
        s_d_overlap = s_d_full[s_d_full.index <= np.datetime64(end_date)]
        # MoM correlation on overlap (need some overlap)
        d_d = mom_logdiff(s_d_overlap)
        common_idx = d_t.index.intersection(d_d.index)
        if common_idx.size < 6:
            continue
        try:
            corr = np.corrcoef(d_t.loc[common_idx], d_d.loc[common_idx])[0,1]
        except Exception:
            continue
        prof_d = month_profile(s_d_overlap)
        cos = cosine_sim(prof_t, prof_d)
        if not (np.isfinite(corr) and np.isfinite(cos)):
            continue
        score = 0.7*corr + 0.3*cos
        out.append((key_d[0], score, corr, cos))
    return out

def cosine_only_candidates(target_s, donor_keys, end_date):
    """Fallback: cosine-only seasonal similarity (no corr filter)."""
    s_t_overlap = target_s[target_s.index <= np.datetime64(end_date)]
    prof_t = month_profile(s_t_overlap)

    out = []
    for _, r in donor_keys.iterrows():
        key_d = (r["Commodity"], r["flow"])
        s_d_full = donors_series.get(key_d)
        if s_d_full is None or s_d_full.empty:
            continue
        s_d_overlap = s_d_full[s_d_full.index <= np.datetime64(end_date)]
        prof_d = month_profile(s_d_overlap)
        cos = cosine_sim(prof_t, prof_d)
        if np.isfinite(cos):
            out.append((key_d[0], cos, np.nan, cos))  # score=cos, corr=nan, cos=cos
    return out

rows = []
no_donor_ct = 0

for (c,f), s_t in targets_series.items():
    last_year = int(pd.to_datetime(s_t.index).max().year)
    end_date  = pd.Timestamp(f"{last_year}-12-01")
    pfx       = extract_prefix(c)

    # candidate sets (progressively relaxed)
    cand_pf  = strict_panel.loc[(strict_panel["flow"]==f) & (strict_panel["prefix"]==pfx),
                                ["Commodity","flow"]].drop_duplicates()
    cand_f   = strict_panel.loc[strict_panel["flow"]==f, ["Commodity","flow"]].drop_duplicates()

    # score with corr+cos, then filter/relax thresholds
    scored = score_candidates(s_t, cand_pf, end_date)
    scored = sorted(scored, key=lambda x: x[1], reverse=True)
    scored1 = [x for x in scored if x[2] >= MIN_CORR1]

    if not scored1:
        scored2 = [x for x in scored if x[2] >= MIN_CORR2]
    else:
        scored2 = scored1

    # if still empty, try broader pool (same flow, any prefix)
    if not scored2:
        scored_broad = score_candidates(s_t, cand_f, end_date)
        scored_broad = sorted(scored_broad, key=lambda x: x[1], reverse=True)
        scored2 = [x for x in scored_broad if x[2] >= MIN_CORR2]

    # if still empty, cosine-only fallback (same flow)
    if not scored2:
        scored_cos = cosine_only_candidates(s_t, cand_f, end_date)
        scored2 = sorted(scored_cos, key=lambda x: x[1], reverse=True)

    # record results (may still be empty)
    if not scored2:
        rows.append([c, f, last_year, None, None, None])
        no_donor_ct += 1
        continue

    scored2 = scored2[:TOP_K]
    donors_str = "; ".join([f"{d}|{s:.3f}|{r if np.isfinite(r) else np.nan:.3f}|{co:.3f}"
                            for d,s,r,co in scored2])
    best = scored2[0]
    rows.append([c, f, last_year, donors_str, best[1], best[2]])

donor_rank = pd.DataFrame(rows, columns=[
    "Commodity","flow","last_year","topK_donors","best_score","best_corr"
])

print("Targets with ≥1 donor after fallbacks:",
      donor_rank["topK_donors"].notna().sum(), "/", donor_rank.shape[0])
print("Targets with no donors (even after cosine-only):", no_donor_ct)

display(donor_rank.head(12))




Targets with ≥1 donor after fallbacks: 92 / 92
Targets with no donors (even after cosine-only): 0


,Commodity,flow,last_year,topK_donors,best_score,best_corr
0,11211 Cattle,export,2021,1121 Cattle|1.000|1.000|1.000; 112 Livestock &...,1.000000,1.000000
1,11211 Cattle,import,2021,1121 Cattle|1.000|1.000|1.000; 111211 Potatoes...,1.000000,1.000000
2,11211X Cattle,export,2021,1121 Cattle|1.000|1.000|1.000; 112 Livestock &...,1.000000,1.000000
3,11211X Cattle,import,2021,1121 Cattle|1.000|1.000|1.000; 111211 Potatoes...,1.000000,1.000000
4,212111 Lignite,export,2022,32532 Pesticides & Other Agricultural Chemical...,0.480566,0.293225
5,212111 Lignite,import,2022,33311 Agricultural Implements|0.423|0.315|0.67...,0.422872,0.314979
6,212112 Coal (excl Anthracite) & Petroleum Gases,export,2022,2121 Coal & Petroleum Gases|0.999|0.999|0.999;...,0.998948,0.998748
7,212112 Coal (excl Anthracite) & Petroleum Gases,import,2022,2121 Coal & Petroleum Gases|0.968|0.965|0.974;...,0.967600,0.964903
8,212299 All Other Metal Ores,export,2022,21229 Other Metal Ores|0.998|0.997|0.999; 2121...,0.997539,0.996922
9,212299 All Other Metal Ores,import,2022,21229 Other Metal Ores|0.867|0.899|0.791; 2122...,0.866686,0.899088


No we bridge-to-2023 with donor index then do ETS for 2024 then fill submission

We use each target’s selected donor list to bridge missing years up to 2023-12
Then, we fit ETS(log-add, m=12, damped) on the bridged level series and forecast 2024 (12 months)
Lastly, we convert to wide (Pred_Export_USD / Pred_Import_USD) and update submission in place, filling only NaNs for Year=2024.

In [49]:
# Ridge with simple numpy (lambda * I regularization)
def ridge_fit(X, y, lam=1.0):
    # Solve (X'X + lam*I) w = X'y
    X = np.asarray(X, float)
    y = np.asarray(y, float)
    n_feat = X.shape[1]
    A = X.T @ X + lam * np.eye(n_feat)
    b = X.T @ y
    try:
        w = np.linalg.solve(A, b)
    except np.linalg.LinAlgError:
        w = np.linalg.pinv(A) @ b
    return w

# Parse donors list like "DonorCommodity|score|corr|cos; ..."
def parse_donor_names(cell):
    if not isinstance(cell, str) or not cell.strip():
        return []
    out = []
    for part in cell.split(";"):
        p = part.strip()
        if not p:
            continue
        name = p.split("|")[0].strip()
        if name:
            out.append(name)
    # deduplicate keeping order
    seen = set(); res=[]
    for n in out:
        if n not in seen:
            seen.add(n); res.append(n)
    return res

def build_mom_matrix_for_period(donor_names, flow, year_start, year_end):
    result = {}
    for y in range(year_start, year_end+1):
        for m in range(1,13):
            result[(y,m)] = []
    # fill from donor MoM
    for dn in donor_names:
        s = donors_series.get((dn, flow))
        if s is None or s.empty:
            # push NaNs so alignment stays consistent
            for y in range(year_start, year_end+1):
                for m in range(1,13):
                    result[(y,m)].append(np.nan)
            continue
        d = mom_logdiff(s)  # indexed by dates (end-of-diff month)
        for y in range(year_start, year_end+1):
            for m in range(1,13):
                dt = pd.Timestamp(y, m, 1)
                result[(y,m)].append(d.get(dt, np.nan))
    # convert each (y,m) list to np.array
    for k in result:
        result[k] = np.asarray(result[k], float)
    return result

def bridge_years_with_donors(target_s, donor_names, flow, last_year):

    # observed log-levels
    ts = pd.Series(target_s.values, index=pd.to_datetime(target_s.index))
    ylog_obs = np.log1p(np.clip(ts, 0, None))

    # overlap for fitting: use MoM diffs on all months up to last_year
    d_t_full = mom_logdiff(ts[ts.index <= pd.Timestamp(f"{last_year}-12-01")])

    # donor MoM matrix for overlap (2015..last_year)
    ov_start_year = int(pd.to_datetime(ts.index.min()).year) + 1  # because diffs start one month after first obs
    X_overlap = []
    y_overlap = []
    donor_mom_ov = build_mom_matrix_for_period(donor_names, flow, ov_start_year, last_year)
    for y in range(ov_start_year, last_year+1):
        for m in range(1,13):
            dt = pd.Timestamp(y, m, 1)
            if dt not in d_t_full.index:
                continue
            x = donor_mom_ov[(y,m)]
            if not np.isfinite(d_t_full.get(dt, np.nan)):
                continue
            # require at least some finite donors for this row
            if np.isfinite(x).sum() < 2:
                continue
            X_overlap.append(np.nan_to_num(x, nan=0.0))
            y_overlap.append(float(d_t_full.get(dt)))
    X_overlap = np.vstack(X_overlap) if len(X_overlap) else None
    y_overlap = np.array(y_overlap) if len(y_overlap) else None

    # fit ridge if we have enough rows; else equal-weights fallback
    if X_overlap is not None and X_overlap.shape[0] >= 24:  # ~2 years of MoM for stability
        w = ridge_fit(X_overlap, y_overlap, lam=1.0)
    else:
        # equal weights across donors that exist
        n = len(donor_names)
        w = np.ones(n) / max(n, 1)

    # now synthesize for years last_year+1..2023
    synth = []
    cur_last = ylog_obs.index.max()
    y_last = float(ylog_obs.loc[cur_last])  # log level at last observed month
    # donor MoM for future years
    if last_year < 2023:
        donor_mom_future = build_mom_matrix_for_period(donor_names, flow, last_year+1, 2023)
        for y in range(last_year+1, 2023+1):
            for m in range(1,13):
                x = donor_mom_future[(y,m)]
                if np.isfinite(x).sum() < 2:
                    # weak donor signal: fallback to seasonal mean-of-month (on observed)
                    prof = month_profile(ts[ts.index <= pd.Timestamp(f"{last_year}-12-01")])
                    d_hat = 0.0  # no drift; rely on ETS later. Keep level.
                else:
                    x = np.nan_to_num(x, nan=0.0)
                    d_hat = float(x @ w)
                y_last = y_last + d_hat  # update log-level
                synth.append((pd.Timestamp(y, m, 1), float(np.expm1(y_last).clip(min=0.0))))
    # combine observed + synthetic
    if synth:
        df_syn = pd.DataFrame(synth, columns=["date","value"])
        base = pd.DataFrame({"date": ylog_obs.index, "value": np.expm1(ylog_obs.values)})
        full = pd.concat([base, df_syn], ignore_index=True).drop_duplicates("date").sort_values("date")
        return pd.Series(full["value"].values, index=full["date"].values)
    else:
        # nothing to bridge
        return pd.Series(np.expm1(ylog_obs.values), index=ylog_obs.index)

def ets_forecast_2024_from_series(level_series):
    """Fit ETS(log-add seasonal, damped) on provided level series (through 2023-12),
       forecast 12 months 2024 in USD levels."""
    s = pd.Series(level_series.values, index=pd.to_datetime(level_series.index)).sort_index()
    y = np.log1p(np.clip(s, 0, None))
    # require sufficient data; otherwise seasonal-naive fallback
    if y.notna().sum() < 18:
        # fallback: repeat month-of-year last observed
        vals = []
        last_year = s.index.max().year
        # try last-year seasonal pattern
        for m in range(1,13):
            sel = s[(s.index.year==last_year) & (s.index.month==m)]
            if not sel.empty:
                vals.append(float(sel.iloc[-1]))
            else:
                vals.append(float(s.iloc[-1]))
        return pd.date_range("2024-01-01", periods=12, freq="MS"), np.array(vals, float)
    try:
        mdl = ExponentialSmoothing(
            y.values, trend='add', seasonal='add', damped_trend=True,
            seasonal_periods=12, initialization_method='estimated'
        ).fit(optimized=True, use_brute=False)
        f = np.expm1(mdl.forecast(12)).clip(min=0.0)
        dates = pd.date_range("2024-01-01", periods=12, freq="MS")
        return dates, f
    except Exception:
        # robust fallback: seasonal-naive (month-of-year from last available year)
        vals = []
        last_year = s.index.max().year
        for m in range(1,13):
            sel = s[(s.index.year==last_year) & (s.index.month==m)]
            vals.append(float(sel.iloc[-1]) if not sel.empty else float(s.iloc[-1]))
        return pd.date_range("2024-01-01", periods=12, freq="MS"), np.array(vals, float)

# Build predictions for all ends_early targets using donor-index bridge + ETS 
pred_rows = []
# map donor_rank: commodity+flow -> donor list, last_year
info_map = donor_rank.set_index(["Commodity","flow"])[["topK_donors","last_year"]].to_dict("index")

# targets_series/ donors_series dicts are already in memory from your previous step
for (c,f), s_t in targets_series.items():
    # Only process those that are in ends_early donor_rank (guard)
    if (c,f) not in donor_rank.set_index(["Commodity","flow"]).index:
        continue
    meta = info_map.get((c,f))
    if meta is None:
        continue
    donor_list = parse_donor_names(meta["topK_donors"])
    last_y = int(meta["last_year"])
    # Bridge missing years to 2023
    bridged = bridge_years_with_donors(s_t, donor_list, f, last_y)
    # Forecast 2024 via ETS
    dts, vals = ets_forecast_2024_from_series(bridged)
    for dt, v in zip(dts, vals):
        pred_rows.append((c, f, dt, float(v)))

pred_2024_ends = pd.DataFrame(pred_rows, columns=["Commodity","flow","date","pred_usd"])
print("ends_early bridged+ETS 2024 predictions:", pred_2024_ends.shape)

# LAST Fill submission

# Ensure dtypes
submission["Year"] = submission["Year"].astype(int)
submission["Month"] = pd.to_numeric(submission["Month"], errors="coerce").astype(int)

# Tidy preds to wide export/import
tidy = (pred_2024_ends
        .assign(Year=2024, Month=lambda d: d["date"].dt.month)
        [["Commodity","Year","Month","flow","pred_usd"]])

expE = (tidy[tidy["flow"]=="export"]
        [["Commodity","Year","Month","pred_usd"]]
        .rename(columns={"pred_usd":"Pred_Export_USD"}))

impE = (tidy[tidy["flow"]=="import"]
        [["Commodity","Year","Month","pred_usd"]]
        .rename(columns={"pred_usd":"Pred_Import_USD"}))

# Count missing before
n_before = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()

# Merge export fills and apply to NaNs only
submission = submission.merge(expE, on=["Commodity","Year","Month"], how="left", suffixes=("", "_END"))
mask_e = submission["Pred_Export_USD"].isna() & submission["Pred_Export_USD_END"].notna()
submission.loc[mask_e, "Pred_Export_USD"] = submission.loc[mask_e, "Pred_Export_USD_END"]
submission.drop(columns=["Pred_Export_USD_END"], inplace=True)

# Merge import fills and apply to NaNs only
submission = submission.merge(impE, on=["Commodity","Year","Month"], how="left", suffixes=("", "_END"))
mask_i = submission["Pred_Import_USD"].isna() & submission["Pred_Import_USD_END"].notna()
submission.loc[mask_i, "Pred_Import_USD"] = submission.loc[mask_i, "Pred_Import_USD_END"]
submission.drop(columns=["Pred_Import_USD_END"], inplace=True)

# Report
filled_new = int(mask_e.sum() + mask_i.sum())
n_after = submission[["Pred_Export_USD","Pred_Import_USD"]].isna().sum()
print(f"Filled cells (ends_early via donor-bridge+ETS): {filled_new}")
print("Remaining NA — export:", int(n_after["Pred_Export_USD"]),
      "| import:", int(n_after["Pred_Import_USD"]))

ends_early bridged+ETS 2024 predictions: (1104, 4)
Filled cells (ends_early via donor-bridge+ETS): 1104
Remaining NA — export: 0 | import: 0


## Final Submission

In [50]:
# Round to cents
for c in ["Pred_Export_USD", "Pred_Import_USD"]:
    submission[c] = pd.to_numeric(submission[c], errors="coerce").round(2)

# Final save
submission.to_csv("Commodities_Prediction_Submission.csv", index=False)
